# Kanren - Estudo de programação lógica

In [102]:
from kanren import Relation, fact, facts, run, var, eq, lall, lany

miniKanren é uma biblioteca para programação relacional, projetada para criar relações bidirecionais (ou seja, você pode tratar entradas e saídas de forma intercambiável). É minimalista e foi originalmente implementada em Scheme, mas hoje está disponível em Python como o pacote

## var, run e eq

`var` é uma classe cujo objeto se declara como variável lógica. Atribuimos ests objetos a difentes chamadas para deteminar que certas variáveis são diferentes.  
É imperativo criar objetos e não usar a classe diretamente, porque sua alocação na memória é o que conecta as verificações com as lacunas a serem preenchidas.

In [48]:
# Criar uma variável lógica
x = var()

print(type(x))
print(x)

<class 'unification.variable.Var'>
~_14


`eq` é similar ao `lambda`: uma função que empresta a sua lógica a alguma tarefa.  
Ele retorna não um produto de uma função (como True e False, ou algum valor ou string), mas sim uma outra função.

In [49]:
goal = eq(x, 5)

print(type(goal))

<class 'function'>


In [50]:
help(goal)

Help on function goal_eq in module kanren.core:

goal_eq(s)



`run` é uma função que retorna tudo que satisfaça a verificação em seu objetivo (goal).

Na célula abaixo, utilizei `run` para procurar itens que, ocupando o espaço da variável declarada, satisfaçam a verificação lógica.

In [51]:
x = var() # variável lógica, usada tanto na criação da função de verificação quanto na busca do resultado.

resultado = run(1, # número de resultados que queremos obter
                x, # encontre valores para x que
                eq(x, 5) # satisfaçam a equação 'x equal 5'
                )

print(resultado)  # Saída: (5,)

(5,)


A célula acima exige 1 valor que, atribuido a x, garante que x==5. O único resultado possível seria 5.  
Abaixo, o número de valores retornados exigidos é 0, o que implica em não haver limites.

In [52]:
run(0, x, eq(x, 5))

(5,)

## Relações e Fatos

Relações são objetos que agregam arestas entre dois ou mais objetos.  
Para a maioria das relações, strings são alocadas nas relação.

In [53]:
pai = Relation()

Na linha acima, criamos uma relação e a chamaremos como `pai`.

In [54]:
fact(pai, "Homer", "Bart") # Fato: há uma relação de pai de Homer para Bart
fact(pai, "Homer", "Lisa") # Fato: há uma relação de pai de Homer para Lisa
fact(pai, "Abe", "Homer") # Fato: há uma relação de pai de Abe para Homer

Com `fact`, colocamos dentro da relação `pai` todas as relações cabíveis pelas strings.  
Toda string "Homer" é `pai` de toda string "Bart".  
Toda string "Homer" é `pai` de toda string "Lisa".  
Toda string "Abe" é `pai` de toda string "Homer".

Uma relação não apenas agrega arestas, mas também funciona como verificação lógica para requisições `run`.  
Aquela relação é um universo onde a lógica busca completude.

In [55]:
# Homer é pai quem?
run(0, x, pai("Homer", x))

('Lisa', 'Bart')

In [56]:
# Quem é pai de Homer?
run(0, x, pai(x, "Homer"))

('Abe',)

Nas duas linhas acima, foram feitas duas consultas diferentes usando a mesma string, a mesma variável e a mesma relação, mudando apenas a cardinalidade.

In [57]:
run(0,x, pai(x, "Bart"), pai(x, "Homer"))

()

Aqui, duas condições são fornecidas.  
Todas as condições fornecidas como argumento separadamente são validadas adicionalmente: "E".  
Na linha acima, busca-se uma string que seja seja pai de "Bart" e pai de "Homer", mas nenhum objeto cumpre esses dois quesitos.

In [58]:
y = var()

# Quem é avô de Bart?
run(0, x, pai(y, "Bart"), pai(x, y))

('Abe',)

Usando combinações lógicas e uma variável adicional, é possível encontrar relações multi-arestas.  
Na célula acima, procura-se um objeto x que cumpra esses requisitos: x é mais de y E y é pai de "Bart".  
Ou seja, o pai do pai do Bart, seu avô.

## Múltiplas variáveis

As variáveis X e Y já foram declaradas mais acima, então podemos utilizá-las no nosso próximo exemplo:

In [64]:
z = var()

# Quem são irmãos?
irmãos = run(0, [x,y], pai(z, x), pai(z, y))

print(irmãos)

(['Homer', 'Homer'], ['Lisa', 'Lisa'], ['Bart', 'Lisa'], ['Lisa', 'Bart'], ['Bart', 'Bart'])


Aqui encontramos um problema: miniKanren não possui mais verificação disassociativa. Isso nos deixa com listas de valores suplicadas.  
Por ora, resolveremos isso com a boa e velha lógica do mais puro Python.

In [69]:
irmãos_reais = []
for i in range(len(irmãos)):
    if irmãos[i][0] != irmãos[i][1]:
        irmãos_reais.append(irmãos[i])

print(irmãos_reais)

[['Bart', 'Lisa'], ['Lisa', 'Bart']]


Isso ainda nos deixa com uma repetição de nomes devido à combinação ser diferente, mas isso será tema para outro capítulo.

## Bicondicionalidade e funções

Sabemos que, se X é pai de Y, Y é filho de X.  
Embora seja redundante, isto serve de demonstração de como múltiplas relações podem ser atribuídas simultaneamente.

In [71]:
filho = Relation()

def pai_e_filho(pai_nome, filho_nome,pai_relação=pai,filho_relação=filho):
    fact(pai_relação, pai_nome, filho_nome)
    fact(filho_relação, filho_nome, pai_nome)

pai_e_filho("José", "Jesus")

Essa função associa, simultaneamente, o pai ao filho e o filho ao pai.

In [ ]:
#Quem é pai de Jesus?

run(0, x, pai(x, "Jesus"))

('José',)

In [ ]:
# Jesus é filho de quem?

run(0, x, filho("Jesus",x))

('José',)

Podemos tentar associar irmãos quando um pai é associado a um filho.

In [83]:
irmão = Relation()

def nasce_um_filho(pai_nome, filho_nome,pai_relação=pai,filho_relação=filho):
    fact(pai_relação, pai_nome, filho_nome)
    fact(filho_relação, filho_nome, pai_nome)
    filhos_mais_velhos = run(0, x, pai_relação(pai_nome, x))
    for filho in filhos_mais_velhos:
        if filho != filho_nome:
            fact(irmão, filho, filho_nome)
            fact(irmão, filho_nome, filho)

In [ ]:
# Nasce o primeiro filho:
nasce_um_filho("Paizão", "Filho 1")

run(0, x, irmão("Filho 1", x))
# Sem irmãos ainda.

()

In [91]:
# Nasce o segundo filho:
nasce_um_filho("Paizão", "Filho 2")

print('São irmãos do Filho 2:',run(0, x, irmão("Filho 2", x)))
print('O Filho 2 é irmão de:',run(0, x, irmão(x,"Filho 2")))

print()

print('São irmãos do Filho 1:',run(0, x, irmão("Filho 1", x)))
print('O Filho 1 é irmão de:',run(0, x, irmão(x,"Filho 1")))

São irmãos do Filho 2: ('Filho 3', 'Filho 1')
O Filho 2 é irmão de: ('Filho 3', 'Filho 1')

São irmãos do Filho 1: ('Filho 3', 'Filho 2')
O Filho 1 é irmão de: ('Filho 3', 'Filho 2')


In [90]:
# Nasce o terceiro filho:
nasce_um_filho("Paizão", "Filho 3")

print('São irmãos do Filho 3:',run(0, x, irmão("Filho 3", x)))
print('O Filho 3 é irmão de:',run(0, x, irmão(x,"Filho 3")))

print()

print('São irmãos do Filho 2:',run(0, x, irmão("Filho 2", x)))
print('O Filho 2 é irmão de:',run(0, x, irmão(x,"Filho 2")))

print()

print('São irmãos do Filho 1:',run(0, x, irmão("Filho 1", x)))
print('O Filho 1 é irmão de:',run(0, x, irmão(x,"Filho 1")))

São irmãos do Filho 3: ('Filho 2', 'Filho 1')
O Filho 3 é irmão de: ('Filho 2', 'Filho 1')

São irmãos do Filho 2: ('Filho 3', 'Filho 1')
O Filho 2 é irmão de: ('Filho 3', 'Filho 1')

São irmãos do Filho 1: ('Filho 3', 'Filho 2')
O Filho 1 é irmão de: ('Filho 3', 'Filho 2')


## lany e lall

A função `lany` é usada para combinar múltiplos objetivos em uma lista lógica onde pelo menos um dos objetivos deve ser satisfeito. Em outras palavras, lany representa uma disjunção lógica (OR) de múltiplos objetivos.

In [94]:
run(0, x, lany(eq(x, 1), eq(x, 2), eq(x, 3)))

(1, 2, 3)

Nesta linha, vemos que cada item retornado cumpre ao menos um dos requisitos.

In [103]:
run(0, x, lany(pai(x, 'Bart'), lall(pai(x, y), pai(y, 'Bart'))))

('Homer', 'Abe')